Righnow candidate is : SVR with PCA n_components = 10 with results 0.23706564506631833

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
test.head(5)

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel

In [5]:
train_labels = train.iloc[:,1]
train_feats = train.iloc[:, 2:]

test_labels = test.iloc[:,1]
test_feats = test.iloc[:, 1:]

In [6]:
train_feats_encode = []
train_feats_encode.append(train.iloc[:, 0])
for i in range(train_feats.shape[1]):
    arr = train_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    train_feats_encode.append(arr)
train_feats_encode = np.array(train_feats_encode).T

test_feats_encode = []
test_feats_encode.append(test.iloc[:, 0])
for i in range(test_feats.shape[1]):
    arr = test_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    test_feats_encode.append(arr)
test_feats_encode = np.array(test_feats_encode).T

*** Keras Deep Learning ***

In [8]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
# define custom R2 metrics for Keras backend
from keras import backend as K
# to tune the NN
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
act_func = 'tanh'

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

Using TensorFlow backend.


In [ ]:
def model():
    model = Sequential()
    # Input layer with dimension input_dims and hidden layer i with input_dims neurons. 
    model.add(Dense(input_dims, input_dim=input_dims, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Activation("linear"))
    # Hidden layer
    model.add(Dense(input_dims, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Activation("linear"))
    # Hidden layer
    model.add(Dense(input_dims, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Activation("linear"))
    # Hidden layer
    model.add(Dense(input_dims, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Activation("linear"))
    # Hidden layer
    model.add(Dense(input_dims//2, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Activation("linear"))
    # Output Layer.
    model.add(Dense(1))
    # Use a large learning rate with decay and a large momentum. 
    # Increase your learning rate by a factor of 10 to 100 and use a high momentum value of 0.9 or 0.99
    # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    # adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='rmsprop',
                  metrics=[r2_keras,"mse"] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [29]:
def train_dl(train_X, eval_X, train_y, eval_y):
    input_dims = train_X.shape[1]
    estimator = KerasRegressor(
        build_fn=model, 
        nb_epoch=300, 
        batch_size=20,
        verbose=1
    )
    estimator.fit(
        train_X, 
        train_y, 
        shuffle=True,
        validation_data=(eval_X, eval_y),
    )
    prediction_train  = estimator.predict(train_X)
    score = r2_score(train_y, prediction_train)
    print("\ntrain : ", score) 
    if eval_X is not None and eval_y is not none:
        prediction_eval = estimator.predict(eval_X)
        score = r2_score(eval_y, prediction_eval)
        print("\ntest : ", score)     
    return estimator, score

In [30]:
train_X, eval_X, train_y, eval_y = train_test_split(train_feats_encode, train_labels, random_state=42)
train_dl(train_X, eval_X, train_y, eval_y)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_41 (Dense)                 (None, 377)           142506      dense_input_9[0][0]              
____________________________________________________________________________________________________
batchnormalization_25 (BatchNorma(None, 377)           754         dense_41[0][0]                   
____________________________________________________________________________________________________
activation_25 (Activation)       (None, 377)           0           batchnormalization_25[0][0]      
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 377)           0           activation_25[0][0]              
___________________________________________________________________________________________

3156/3156 [==============================] - 4s - loss: 156.3863 - r2_keras: -0.0334 - val_loss: 156.7013 - val_r2_keras: -0.0457
Epoch 36/300
3156/3156 [==============================] - 3s - loss: 150.9913 - r2_keras: 0.0086 - val_loss: 166.7905 - val_r2_keras: -0.1099
Epoch 37/300
3156/3156 [==============================] - 3s - loss: 147.5514 - r2_keras: 0.0243 - val_loss: 150.9727 - val_r2_keras: -0.0271
Epoch 38/300
3156/3156 [==============================] - 3s - loss: 138.0057 - r2_keras: 0.1213 - val_loss: 161.3844 - val_r2_keras: -0.0641
Epoch 39/300
3156/3156 [==============================] - 4s - loss: 131.0163 - r2_keras: 0.1502 - val_loss: 168.6712 - val_r2_keras: -0.1845
Epoch 40/300
3156/3156 [==============================] - 4s - loss: 125.7380 - r2_keras: 0.2068 - val_loss: 178.7100 - val_r2_keras: -0.2712
Epoch 41/300
3156/3156 [==============================] - 3s - loss: 113.1340 - r2_keras: 0.2771 - val_loss: 97.0453 - val_r2_keras: 0.3986
Epoch 42/300
3156/31

3156/3156 [==============================] - 5s - loss: 77.9375 - r2_keras: 0.5337 - val_loss: 130.9458 - val_r2_keras: 0.0781
Epoch 94/300
3156/3156 [==============================] - 5s - loss: 78.0555 - r2_keras: 0.5420 - val_loss: 75.1767 - val_r2_keras: 0.5472
Epoch 95/300
3156/3156 [==============================] - 4s - loss: 79.4748 - r2_keras: 0.5220 - val_loss: 74.7850 - val_r2_keras: 0.5634
Epoch 96/300
3156/3156 [==============================] - 5s - loss: 77.3195 - r2_keras: 0.5356 - val_loss: 81.3127 - val_r2_keras: 0.5120
Epoch 97/300
3156/3156 [==============================] - 5s - loss: 78.2833 - r2_keras: 0.5376 - val_loss: 92.6286 - val_r2_keras: 0.4383
Epoch 98/300
3156/3156 [==============================] - 5s - loss: 80.3506 - r2_keras: 0.5116 - val_loss: 74.0485 - val_r2_keras: 0.5620
Epoch 99/300
3156/3156 [==============================] - 5s - loss: 78.5517 - r2_keras: 0.5260 - val_loss: 79.2833 - val_r2_keras: 0.5310
Epoch 100/300
3156/3156 [==============

3156/3156 [==============================] - 3s - loss: 75.0924 - r2_keras: 0.5477 - val_loss: 122.4946 - val_r2_keras: 0.2228
Epoch 151/300
3156/3156 [==============================] - 3s - loss: 76.1957 - r2_keras: 0.5476 - val_loss: 98.7135 - val_r2_keras: 0.3967
Epoch 152/300
3156/3156 [==============================] - 3s - loss: 72.6654 - r2_keras: 0.5740 - val_loss: 69.0378 - val_r2_keras: 0.5913
Epoch 153/300
3156/3156 [==============================] - 3s - loss: 74.5055 - r2_keras: 0.5503 - val_loss: 71.9246 - val_r2_keras: 0.56050.
Epoch 154/300
3156/3156 [==============================] - 3s - loss: 73.5962 - r2_keras: 0.5635 - val_loss: 68.9852 - val_r2_keras: 0.5890
Epoch 155/300
3156/3156 [==============================] - 3s - loss: 73.8041 - r2_keras: 0.5643 - val_loss: 78.4221 - val_r2_keras: 0.5421
Epoch 156/300
3156/3156 [==============================] - 3s - loss: 73.1522 - r2_keras: 0.5731 - val_loss: 73.1234 - val_r2_keras: 0.5720
Epoch 157/300
3156/3156 [======

3156/3156 [==============================] - 4s - loss: 74.0064 - r2_keras: 0.5615 - val_loss: 79.2625 - val_r2_keras: 0.5216
Epoch 209/300
3156/3156 [==============================] - 4s - loss: 71.9908 - r2_keras: 0.5852 - val_loss: 72.2320 - val_r2_keras: 0.5687
Epoch 210/300
3156/3156 [==============================] - 4s - loss: 72.2060 - r2_keras: 0.5701 - val_loss: 71.3537 - val_r2_keras: 0.5842
Epoch 211/300
3156/3156 [==============================] - 4s - loss: 73.0838 - r2_keras: 0.5716 - val_loss: 83.1810 - val_r2_keras: 0.5055
Epoch 212/300
3156/3156 [==============================] - 4s - loss: 73.1333 - r2_keras: 0.5661 - val_loss: 78.4603 - val_r2_keras: 0.5269
Epoch 213/300
3156/3156 [==============================] - 4s - loss: 74.5343 - r2_keras: 0.5542 - val_loss: 88.7776 - val_r2_keras: 0.4688
Epoch 214/300
3156/3156 [==============================] - 4s - loss: 74.0295 - r2_keras: 0.5610 - val_loss: 269.2692 - val_r2_keras: -0.8561
Epoch 215/300
3156/3156 [=======

3156/3156 [==============================] - 3s - loss: 72.1721 - r2_keras: 0.5726 - val_loss: 69.7820 - val_r2_keras: 0.5823
Epoch 267/300
3156/3156 [==============================] - 4s - loss: 72.3214 - r2_keras: 0.5752 - val_loss: 70.4868 - val_r2_keras: 0.5807
Epoch 268/300
3156/3156 [==============================] - 4s - loss: 74.1890 - r2_keras: 0.5417 - val_loss: 70.6584 - val_r2_keras: 0.5787
Epoch 269/300
3156/3156 [==============================] - 3s - loss: 71.6955 - r2_keras: 0.5659 - val_loss: 74.4558 - val_r2_keras: 0.5431
Epoch 270/300
3156/3156 [==============================] - 4s - loss: 73.8683 - r2_keras: 0.5485 - val_loss: 70.5446 - val_r2_keras: 0.5703
Epoch 271/300
3156/3156 [==============================] - 3s - loss: 72.5191 - r2_keras: 0.5666 - val_loss: 73.2747 - val_r2_keras: 0.5716
Epoch 272/300
3156/3156 [==============================] - 3s - loss: 72.2101 - r2_keras: 0.5757 - val_loss: 84.9679 - val_r2_keras: 0.4628
Epoch 273/300
3156/3156 [=========

NameError: global name 'none' is not defined

In [ ]:
test_predictions = estimator.predict(test_feats_encode)

In [ ]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': test_predictions})
output.to_csv('results/keras/[%s][%s].csv' % (str(model.__class__.__name__),score), index=False)
